In [ ]:
# --- Setup & Configuration (copied from 02_analysis.ipynb) ---
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

PROJECT_ROOT = Path(r"C:\Users\kobyw\OneDrive\Documents\nba-pace-analytics").resolve()
PROC_DIR = PROJECT_ROOT / "data" / "processed"

# Load processed data
all_logs_df = pd.read_csv(PROC_DIR / "team_game_logs_with_metrics_2024_2026.csv")
team_outcomes_df = pd.read_csv(PROC_DIR / "team_season_outcomes_2024_2026.csv")

# Split data by season
log_2026 = all_logs_df[all_logs_df['SEASON'] == '2025-26'].copy()
log_2025 = all_logs_df[all_logs_df['SEASON'] == '2024-25'].copy()
log_2024 = all_logs_df[all_logs_df['SEASON'] == '2023-24'].copy()

wins_2026 = team_outcomes_df[team_outcomes_df['SEASON'] == '2025-26'].copy()
wins_2025 = team_outcomes_df[team_outcomes_df['SEASON'] == '2024-25'].copy()
wins_2024 = team_outcomes_df[team_outcomes_df['SEASON'] == '2023-24'].copy()

MIN_GAMES_THRESHOLD = 0

def calculate_team_metric(game_logs_df: pd.DataFrame, metric_col: str, min_threshold: int = MIN_GAMES_THRESHOLD) -> pd.Series:
    """Return season-average metric by TEAM_NAME."""
    return (
        game_logs_df[game_logs_df["MIN"] > min_threshold]
        .groupby("TEAM_NAME")[metric_col]
        .mean()
    )

def make_abbrev(name):
    """Generate team abbreviation from team name (fallback)."""
    if not isinstance(name, str) or len(name) == 0:
        return ''
    parts = name.split()
    if len(parts) >= 2:
        return (''.join(p[0] for p in parts)[:3]).upper()
    return name[:3].upper()

## Extra Coding Not Fit for the Analysis

Here sits some extra expirement I attempted takinf PHX, DAL, IND out of my expirement due to them losing thier best best offensive players from the 2024-2025 Season to the 2025-2026 Season

In [ ]:
# -------------------- Sensitivity Analysis: Remove PHX, DAL, IND --------------------
# These teams lost key offensive players; let's see if they skew the correlations

teams_to_exclude = ['Phoenix Suns', 'Dallas Mavericks', 'Indiana Pacers']

# Filter analysis_df to exclude these teams
analysis_df_filtered = analysis_df[~analysis_df.index.isin(teams_to_exclude)].copy()

print("="*70)
print("SENSITIVITY ANALYSIS: Removing PHX, DAL, IND")
print("="*70)
print(f"\nOriginal sample size: {len(analysis_df)}")
print(f"Filtered sample size: {len(analysis_df_filtered)}")
print(f"Teams removed: {teams_to_exclude}")

# Recalculate correlations WITHOUT these three teams
print("\n" + "="*70)
print("CORRELATIONS (2025→2026) - EXCLUDING PHX, DAL, IND")
print("="*70)

wins_pace_corr_filt = analysis_df_filtered['WIN_PCT_DELTA_26_25'].corr(analysis_df_filtered['PACE_DELTA'])
wins_ortg_corr_filt = analysis_df_filtered['WIN_PCT_DELTA_26_25'].corr(analysis_df_filtered['ORTG_DELTA'])
wins_ts_corr_filt = analysis_df_filtered['WIN_PCT_DELTA_26_25'].corr(analysis_df_filtered['TS_DELTA'])

pm_pace_corr_filt = analysis_df_filtered['PM_PG_DELTA_26_25'].corr(analysis_df_filtered['PACE_DELTA'])
pm_ortg_corr_filt = analysis_df_filtered['PM_PG_DELTA_26_25'].corr(analysis_df_filtered['ORTG_DELTA'])
pm_ts_corr_filt = analysis_df_filtered['PM_PG_DELTA_26_25'].corr(analysis_df_filtered['TS_DELTA'])

print("\nWin Percentage Correlations:")
print(f"  PACE Δ vs WIN% Δ: {wins_pace_corr_filt:7.3f}  (was {wins_pace_corr:7.3f}, Δ {wins_pace_corr_filt - wins_pace_corr:+.3f})")
print(f"  ORTG Δ vs WIN% Δ: {wins_ortg_corr_filt:7.3f}  (was {wins_ortg_corr:7.3f}, Δ {wins_ortg_corr_filt - wins_ortg_corr:+.3f})")
print(f"  TS% Δ  vs WIN% Δ: {wins_ts_corr_filt:7.3f}  (was {wins_ts_corr:7.3f}, Δ {wins_ts_corr_filt - wins_ts_corr:+.3f})")

print("\nPoint Differential Per Game Correlations:")
print(f"  PACE Δ vs PM/G Δ: {pm_pace_corr_filt:7.3f}  (was {pm_pace_corr:7.3f}, Δ {pm_pace_corr_filt - pm_pace_corr:+.3f})")
print(f"  ORTG Δ vs PM/G Δ: {pm_ortg_corr_filt:7.3f}  (was {pm_ortg_corr:7.3f}, Δ {pm_ortg_corr_filt - pm_ortg_corr:+.3f})")
print(f"  TS% Δ  vs PM/G Δ: {pm_ts_corr_filt:7.3f}  (was {pm_ts_corr:7.3f}, Δ {pm_ts_corr_filt - pm_ts_corr:+.3f})")

print("="*70)

# Show the excluded teams' data for context
print("\nExcluded Teams' Performance (2025→2026):")
excluded_data = analysis_df[analysis_df.index.isin(teams_to_exclude)][
    ['PACE_DELTA', 'ORTG_DELTA', 'TS_DELTA', 'WIN_PCT_DELTA_26_25', 'PM_PG_DELTA_26_25', 'ABBREV']
].round(3)
print(excluded_data.to_string())

# Interpretation
print("\n" + "="*70)
print("INTERPRETATION")
print("="*70)
pace_change = wins_pace_corr_filt - wins_pace_corr
ortg_change = wins_ortg_corr_filt - wins_ortg_corr

if abs(pace_change) > 0.10:
    print(f"⚠️  PACE correlation changed by {pace_change:+.3f} — These teams ARE outliers")
else:
    print(f"✓ PACE correlation stable (change: {pace_change:+.3f}) — These teams fit the trend")

if abs(ortg_change) > 0.10:
    print(f"⚠️  ORTG correlation changed by {ortg_change:+.3f} — Efficiency impact significant")
else:
    print(f"✓ ORTG correlation stable (change: {ortg_change:+.3f}) — Efficiency findings robust")

In [ ]:
# -------------------- Visualization: Sensitivity Analysis Comparison --------------------

# Create a comparison chart
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left plot: PACE & ORTG correlations comparison
metrics = ['PACE vs\nWIN%', 'ORTG vs\nWIN%', 'TS% vs\nWIN%']
before = [wins_pace_corr, wins_ortg_corr, wins_ts_corr]
after = [wins_pace_corr_filt, wins_ortg_corr_filt, wins_ts_corr_filt]

x = np.arange(len(metrics))
width = 0.35

ax1 = axes[0]
bars1 = ax1.bar(x - width/2, before, width, label='All 30 Teams', alpha=0.8, color='steelblue')
bars2 = ax1.bar(x + width/2, after, width, label='Excluding PHX/DAL/IND (27 Teams)', alpha=0.8, color='coral')

ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax1.set_ylabel('Correlation Coefficient', fontsize=11, fontweight='bold')
ax1.set_title('Win % Correlations: Before vs After Exclusion', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(metrics)
ax1.legend()
ax1.set_ylim(-0.6, 0.6)
ax1.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom' if height >= 0 else 'top', fontsize=9)

# Right plot: Scatter comparison
ax2 = axes[1]

# Plot all 30 teams
ax2.scatter(analysis_df['PACE_DELTA'], analysis_df['ORTG_DELTA'], 
           s=100, alpha=0.6, color='steelblue', label='All Teams', edgecolors='black', linewidth=0.5)

# Highlight the 3 excluded teams
excluded = analysis_df[analysis_df.index.isin(teams_to_exclude)]
ax2.scatter(excluded['PACE_DELTA'], excluded['ORTG_DELTA'], 
           s=150, alpha=0.9, color='red', label='Excluded (PHX/DAL/IND)', 
           marker='X', edgecolors='darkred', linewidth=1)

# Add trend lines
z_all = np.polyfit(analysis_df['PACE_DELTA'].dropna(), analysis_df['ORTG_DELTA'].dropna(), 1)
p_all = np.poly1d(z_all)
z_filt = np.polyfit(analysis_df_filtered['PACE_DELTA'].dropna(), analysis_df_filtered['ORTG_DELTA'].dropna(), 1)
p_filt = np.poly1d(z_filt)

x_line = np.linspace(analysis_df['PACE_DELTA'].min(), analysis_df['PACE_DELTA'].max(), 100)
ax2.plot(x_line, p_all(x_line), 'steelblue', linestyle='--', linewidth=2, alpha=0.7, label=f'Trend (All): r={pace_ortg_corr:.3f}')
ax2.plot(x_line, p_filt(x_line), 'coral', linestyle='--', linewidth=2, alpha=0.7, label=f'Trend (Filtered): r={analysis_df_filtered["PACE_DELTA"].corr(analysis_df_filtered["ORTG_DELTA"]):.3f}')

ax2.axhline(y=0, color='gray', linestyle=':', alpha=0.5)
ax2.axvline(x=0, color='gray', linestyle=':', alpha=0.5)
ax2.set_xlabel('Pace Change (YoY)', fontsize=11, fontweight='bold')
ax2.set_ylabel('ORTG Change (YoY)', fontsize=11, fontweight='bold')
ax2.set_title('Pace vs Efficiency: Impact of 3-Team Removal', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Label the excluded teams
for team in excluded.index:
    ax2.annotate(excluded.loc[team, 'ABBREV'],
                (excluded.loc[team, 'PACE_DELTA'], excluded.loc[team, 'ORTG_DELTA']),
                xytext=(5, 5), textcoords='offset points', fontsize=10, fontweight='bold', color='red')

plt.tight_layout()
plt.show()

print("\n📊 Visualization complete!")
print(f"   Red X markers = PHX, DAL, IND (excluded teams)")
print(f"   Blue trend line = All 30 teams")
print(f"   Orange trend line = 27 teams (without outliers)")